<a href="https://colab.research.google.com/github/TASM-Repo/TASM/blob/main/Conduit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conduit Notebook (Zenodo-Only Edition)

**Purpose:**
- Run TASM simulations in Colab VM  
- Save outputs locally under `/content/data/` & `/content/artifacts/`  
- Hash & manifest every artifact  
- Upload + publish to Zenodo via API, recording DOIs  
- No Google Drive dependency—fully self-contained in Colab  

**Core workflow:**
1. Define local storage paths (`/content/data`, `/content/artifacts`).  
2. Pin environment.  
3. Run CF/GR simulations → write CSVs, plots, etc., to `/content/data`.  
4. Call `log_and_publish(path)` to:
   - copy to `/content/artifacts/`  
   - hash & append to `manifest.json`  
   - create a Zenodo deposition, upload files + metadata, publish → get DOI  
   - write DOI back into the manifest  
5. End of run: `/content/artifacts/manifest.json` has full audit trail.

**Folder structure (in VM):**

In [12]:
import os
# ⚠️ Replace the string below with your actual token (keep it secret!)
os.environ['ZENODO_TOKEN'] = 'RjNnTNy1T6O1Mf6rGkCmtJUTjQLCEKKmmvNrx1dWmNDQM1k0rTLnSxqXpgbj'


In [13]:
import requests, os
resp = requests.get(
    'https://zenodo.org/api/deposit/depositions',
    params={'access_token': os.environ['ZENODO_TOKEN']}
)
print('Zenodo auth status:', resp.status_code)


Zenodo auth status: 200


In [14]:
import os, requests, json, pathlib

# Your token should already be set in os.environ['ZENODO_TOKEN']
ZENODO_API   = 'https://zenodo.org/api/deposit/depositions'
HEADERS      = {"Content-Type": "application/json"}
PARAMS       = {'access_token': os.environ['ZENODO_TOKEN']}

# Load or define your global metadata template
METADATA = {
    "metadata": {
        "title": "TASM Simulation Artifacts v1.0",
        "upload_type": "dataset",
        "description": "Curvature-Factor lensing slice outputs, CF gradients 1.0/2.0/2.999, manifest hash, generated via Conduit notebook.",
        "creators": [{"name": "Tejera, Michael", "affiliation": "Independent Researcher"}],
        "keywords": ["cosmology","CF","lensing","TASM","AI-assisted"],
        "license": "CC-BY-4.0"
    }
}


In [15]:
def create_deposition():
    """Start a new Zenodo deposition and return its ID."""
    r = requests.post(ZENODO_API, params=PARAMS, json={})
    r.raise_for_status()
    dep = r.json()
    print(f"🆔 New deposition id = {dep['id']}")
    return dep['id']

def zenodo_upload(depid, file_path):
    """Upload one file to an existing deposition."""
    path = pathlib.Path(file_path)
    files = {'file': open(path, 'rb')}
    url = f"{ZENODO_API}/{depid}/files"
    r = requests.post(url, params=PARAMS, files=files)
    r.raise_for_status()
    print(f"✅ Uploaded {path.name} to deposition {depid}")

def finalize_deposition(depid):
    """Publish the deposition and return its DOI."""
    url = f"{ZENODO_API}/{depid}/actions/publish"
    r = requests.post(url, params=PARAMS)
    r.raise_for_status()
    doi = r.json()['doi']
    print(f"🎉 Published: DOI = {doi}")
    return doi


In [21]:
def log_and_publish(src_path):
    # 1) Local hashing & manifest
    log_artifact(src_path)

    # 2) Zenodo workflow
    depid = create_deposition()
    zenodo_upload(depid, ARTIFACT_DIR / pathlib.Path(src_path).name)
    doi   = finalize_deposition(depid)

    # 3) Append DOI to manifest entry
    manifest = ARTIFACT_DIR / 'manifest.json'
    data = json.loads(manifest.read_text())
    data[-1]['doi'] = doi
    manifest.write_text(json.dumps(data, indent=2))

    print(f"🔖 Added DOI to manifest entry.")


In [23]:
# ──────────────── Self-Contained Zenodo Smoke-Test ────────────────

import os
import pathlib
import hashlib
import json
import datetime as dt
import requests
import pandas as pd

# 1) Local folder setup
DATA_DIR     = pathlib.Path('/content/data')
ARTIFACT_DIR = pathlib.Path('/content/artifacts')
for d in (DATA_DIR, ARTIFACT_DIR):
    d.mkdir(parents=True, exist_ok=True)

# 2) Zenodo config (ensure ZENODO_TOKEN is already set in env)
ZENODO_TOKEN = os.getenv('ZENODO_TOKEN')
if not ZENODO_TOKEN:
    raise RuntimeError("ZENODO_TOKEN not found in environment!")
ZENODO_API = 'https://zenodo.org/api/deposit/depositions'
HEADERS    = {"Content-Type": "application/json"}
PARAMS     = {'access_token': ZENODO_TOKEN}

METADATA = {
    "metadata": {
        "title": "TASM Simulation Artifacts v1.0",
        "upload_type": "dataset",
        "description": "Curvature-Factor lensing demo, auto-hashed & published via Conduit.",
        "creators": [{"name": "Tejera, Michael", "affiliation": "Independent Researcher"}],
        "keywords": ["cosmology","TASM","CF","lensing","AI-assisted"],
        "license": "CC-BY-4.0"
    }
}

# 3) Helper functions

def sha256(path):
    h = hashlib.sha256()
    with open(path, 'rb') as f:
        for chunk in iter(lambda: f.read(8192), b''):
            h.update(chunk)
    return h.hexdigest()

def log_artifact(path):
    src  = pathlib.Path(path)
    dest = ARTIFACT_DIR / src.name
    dest.write_bytes(src.read_bytes())
    record = {
        'file': dest.name,
        'sha256': sha256(dest),
        'timestamp': dt.datetime.utcnow().isoformat() + 'Z'
    }
    manifest = ARTIFACT_DIR / 'manifest.json'
    data = json.loads(manifest.read_text()) if manifest.exists() else []
    data.append(record)
    manifest.write_text(json.dumps(data, indent=2))
    print(f"📦 Logged locally: {dest.name}")

def create_deposition():
    r = requests.post(ZENODO_API, params=PARAMS, json={})
    r.raise_for_status()
    depid = r.json()['id']
    # set metadata
    r2 = requests.put(f"{ZENODO_API}/{depid}", params=PARAMS, json=METADATA, headers=HEADERS)
    r2.raise_for_status()
    print(f"🆔 Created deposition {depid}")
    return depid

def zenodo_upload(depid, file_path):
    files = {'file': open(file_path, 'rb')}
    url = f"{ZENODO_API}/{depid}/files"
    r = requests.post(url, params=PARAMS, files=files)
    r.raise_for_status()
    print(f"✅ Uploaded {pathlib.Path(file_path).name}")

def finalize_deposition(depid):
    r = requests.post(f"{ZENODO_API}/{depid}/actions/publish", params=PARAMS)
    r.raise_for_status()
    doi = r.json()['doi']
    print(f"🎉 Published DOI = {doi}")
    return doi

def log_and_publish(path):
    log_artifact(path)
    depid = create_deposition()
    zenodo_upload(depid, ARTIFACT_DIR / pathlib.Path(path).name)
    doi = finalize_deposition(depid)
    manifest = ARTIFACT_DIR / 'manifest.json'
    data = json.loads(manifest.read_text())
    data[-1]['doi'] = doi
    manifest.write_text(json.dumps(data, indent=2))
    print(f"🔖 DOI recorded in manifest.")

# 4) Demo file creation and test

demo = DATA_DIR / 'demo_CF_run.csv'
pd.DataFrame({'theta':[0,1,2], 'xi_plus':[0.1,0.2,0.3]}).to_csv(demo, index=False)
print("📝 Demo CSV written:", demo)

# 5) Run the full pipeline
log_and_publish(demo)

# 6) Show final manifest
print("\n📄 Final manifest.json contents:")
print((ARTIFACT_DIR/'manifest.json').read_text())

📝 Demo CSV written: /content/data/demo_CF_run.csv
📦 Logged locally: demo_CF_run.csv
🆔 Created deposition 15367577
✅ Uploaded demo_CF_run.csv
🎉 Published DOI = 10.5281/zenodo.15367577
🔖 DOI recorded in manifest.

📄 Final manifest.json contents:
[
  {
    "file": "demo_CF_run.csv",
    "sha256": "de7bb8d4ac66e4e46d2bfeb635ce271a46e6ac4945b62a37970f5f9f0a2fb9e1",
    "timestamp": "2025-05-08T18:52:23.627628Z",
    "doi": "10.5281/zenodo.15367577"
  }
]
